# Data Analysis on FINRA_TRACE

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

from sklearn import preprocessing

### Read the pandas dataframe we got from data preparation

In [2]:
root_folder = Path('../Data/Pickle/')
file_name = 'TRACE2014_jinming.pkl'
file_path = root_folder / file_name

data = pd.read_pickle(file_path)

In [23]:
root_folder = Path('../Data/Dataset//')
file_name = 'Mergent_FISD_Bonds_Ratings.csv'
file_path = root_folder / file_name

rating_fields = ['COMPLETE_CUSIP','RATING_TYPE', 'RATING_DATE', 'RATING', 'RATING_STATUS' \
                ,'REASON', 'RATING_STATUS_DATE', 'INVESTMENT_GRADE']
rating_dtypes = {'COMPLETE_CUSIP':str,'RATING_TYPE':str, 'RATING':str, 'RATING_STATUS':str \
                ,'REASON':str, 'INVESTMENT_GRADE':str}
rating_parse_dates = ['RATING_DATE','RATING_STATUS_DATE']

ratings_data = pd.read_csv(file_path,usecols=rating_fields, dtype=rating_dtypes \
                           ,parse_dates=rating_parse_dates,infer_datetime_format=True)

In [24]:
ratings_data.head()

,RATING_TYPE,RATING_DATE,RATING,RATING_STATUS,REASON,RATING_STATUS_DATE,INVESTMENT_GRADE,COMPLETE_CUSIP
0,DPR,1989-10-24,NR,NaN,NaN,NaT,NaN,000361AA3
1,FR,1996-01-04,BBB-,NaN,IL,NaT,NaN,000361AA3
2,FR,1997-08-20,BBB,NaN,UPG,NaT,NaN,000361AA3
3,FR,1999-02-01,BBB,NaN,AFRM,NaT,NaN,000361AA3
4,FR,2000-08-16,BBB,NaN,IR,NaT,NaN,000361AA3


In [28]:
ratings_data.loc[ratings_data['COMPLETE_CUSIP'] == '000361AA3']

,RATING_TYPE,RATING_DATE,RATING,RATING_STATUS,REASON,RATING_STATUS_DATE,INVESTMENT_GRADE,COMPLETE_CUSIP
0,DPR,1989-10-24,NR,NaN,NaN,NaT,NaN,000361AA3
1,FR,1996-01-04,BBB-,NaN,IL,NaT,NaN,000361AA3
2,FR,1997-08-20,BBB,NaN,UPG,NaT,NaN,000361AA3
3,FR,1999-02-01,BBB,NaN,AFRM,NaT,NaN,000361AA3
4,FR,2000-08-16,BBB,NaN,IR,NaT,NaN,000361AA3
5,FR,2001-07-11,BBB,NaN,AFRM,NaT,Y,000361AA3
6,MR,1993-05-26,Baa3,NaN,IL,NaT,NaN,000361AA3
7,MR,1997-01-23,Baa2,NaN,UPG,NaT,NaN,000361AA3
8,MR,1999-12-29,Baa2,NaN,IR,NaT,NaN,000361AA3
9,MR,2000-12-15,Baa2,NaN,IR,NaT,NaN,000361AA3


In [3]:
data.head()

,TRD_RPT_DTTM,TRD_EXCTN_DTTM,TRC_ST,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,ENTRD_VOL_QT,RPTD_PR,RPT_SIDE_CD,Report_Dealer_Index,...,document_date,ISSUER_ID,COMPLETE_CUSIP,AGENT_ID,CUSIP_NAME,INDUSTRY_GROUP,INDUSTRY_CODE,PARENT_ID,NAICS_CODE,SIC_CODE
0,2014-07-16 09:47:25,2014-07-16 09:47:13,T,TWIX3666829,982526AB1,CORP,25000.0,102.991997,B,0,...,2014-07-16,38591,982526AB1,52600,WRIGLEY WM JR CO,1,10,56732,31134,2067
1,2014-12-23 16:11:32,2014-12-23 16:11:17,T,WAMU3885939,92936PAB6,CORP,214.0,95.934578,S,83,...,2014-12-23,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035
2,2014-12-05 14:43:44,2014-12-05 14:43:27,T,WAMU3885939,92936PAB6,CORP,100.0,92.500000,S,83,...,2014-12-05,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035
3,2014-12-03 10:48:12,2014-12-03 10:48:11,T,WAMU3885939,92936PAB6,CORP,1.0,88.000000,B,0,...,2014-12-03,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035
4,2014-12-23 16:11:16,2014-12-23 16:10:56,T,WAMU3885939,92936PAB6,CORP,2000.0,95.250000,S,83,...,2014-12-23,44811,92936PAB6,12434,WMI HLDGS CORP,2,20,12434,52212,6035


In [4]:
data.columns

Index(['TRD_RPT_DTTM', 'TRD_EXCTN_DTTM', 'TRC_ST', 'BOND_SYM_ID', 'CUSIP_ID',
       'SCRTY_TYPE_CD', 'ENTRD_VOL_QT', 'RPTD_PR', 'RPT_SIDE_CD',
       'Report_Dealer_Index', 'Contra_Party_Index', 'document_date',
       'ISSUER_ID', 'COMPLETE_CUSIP', 'AGENT_ID', 'CUSIP_NAME',
       'INDUSTRY_GROUP', 'INDUSTRY_CODE', 'PARENT_ID', 'NAICS_CODE',
       'SIC_CODE'],
      dtype='object')

In [4]:
a = data['Report_Dealer_Index'].nunique()
print('How many unique dealer? {}'.format(a))

How many unique dealer? 1221


In [5]:
b = data['document'].nunique()
print('How many document for topic modeling? {}'.format(b))

How many document for topic modeling? 98304


In [6]:
c = b/a
print('Average working days? {}'.format(c))

Average working days? 80.51105651105651


In [7]:
d = data['BOND_SYM_ID'].nunique()
print('How many token for topic modeling? {}'.format(d))

How many token for topic modeling? 12078


In [8]:
matrix1_non_zero_entries = 4072730
e = matrix1_non_zero_entries / b
print('How many token per documnet on average? {}'.format(e))

How many token per documnet on average? 41.429951985677086


In [9]:
print('What is the TRC_ST ditribution?')
data.groupby(['TRC_ST']).size()

What is the TRC_ST ditribution?


TRC_ST
R      83208
T    6239629
dtype: int64

In [10]:
data[['ENTRD_VOL_QT','RPTD_PR']].describe()

,ENTRD_VOL_QT,RPTD_PR
count,6.322837e+06,6.322837e+06
mean,5.920860e+05,1.034367e+02
std,2.400993e+06,1.207773e+01
min,1.000000e-02,0.000000e+00
25%,1.000000e+04,9.960500e+01
50%,3.300000e+04,1.030000e+02
75%,2.500000e+05,1.087440e+02
max,1.216477e+09,2.322500e+02


In [21]:
data.loc[data['BOND_SYM_ID']=='ALGT4137543']

,TRD_RPT_DTTM,TRD_EXCTN_DTTM,TRC_ST,BOND_SYM_ID,CUSIP_ID,SCRTY_TYPE_CD,ENTRD_VOL_QT,RPTD_PR,RPT_SIDE_CD,Report_Dealer_Index,Contra_Party_Index,document_date,document
23924,2014-09-25 15:03:41,25SEP2014 14:03:36,T,ALGT4137543,01748XAA0,CORP,15000.0,105.500,S,5,99999,2014-09-25,"5,2014-09-25"
23949,2014-09-25 15:03:10,25SEP2014 14:03:04,T,ALGT4137543,01748XAA0,CORP,20000.0,105.500,S,5,99999,2014-09-25,"5,2014-09-25"
23952,2014-09-25 15:02:54,25SEP2014 14:02:50,T,ALGT4137543,01748XAA0,CORP,20000.0,105.500,S,5,99999,2014-09-25,"5,2014-09-25"
23980,2014-09-25 15:02:44,25SEP2014 14:02:38,T,ALGT4137543,01748XAA0,CORP,40000.0,105.500,S,5,99999,2014-09-25,"5,2014-09-25"
23982,2014-09-25 15:03:27,25SEP2014 14:03:20,T,ALGT4137543,01748XAA0,CORP,15000.0,105.500,S,5,99999,2014-09-25,"5,2014-09-25"
115901,2014-07-24 13:26:53,24JUL2014 12:26:44,T,ALGT4137543,01748XAA0,CORP,11000.0,102.250,S,151,99999,2014-07-24,"151,2014-07-24"
115902,2014-07-24 13:26:47,24JUL2014 12:26:44,T,ALGT4137543,01748XAA0,CORP,7000.0,102.250,S,151,99999,2014-07-24,"151,2014-07-24"
115903,2014-08-12 11:27:00,12AUG2014 10:26:00,T,ALGT4137543,01748XAA0,CORP,10000.0,104.295,S,151,99999,2014-08-12,"151,2014-08-12"
115904,2014-10-31 11:53:36,31OCT2014 10:53:26,T,ALGT4137543,01748XAA0,CORP,3000.0,102.500,S,151,99999,2014-10-31,"151,2014-10-31"
115905,2014-09-16 16:10:36,16SEP2014 15:10:31,T,ALGT4137543,01748XAA0,CORP,2000.0,104.000,S,151,99999,2014-09-16,"151,2014-09-16"
